# 📘 Agentic Architectures 11: Meta-Controller

Welcome to the eleventh notebook in our series. Today, we're building a **Meta-Controller**, a supervisory agent architecture that orchestrates a team of specialized sub-agents. This pattern is fundamental to creating powerful, multi-talented AI systems.

Instead of building a single, monolithic agent that tries to do everything, the Meta-Controller acts as a smart dispatcher. It receives an incoming request, analyzes its nature, and routes it to the most appropriate specialist from a pool of available agents. This allows each sub-agent to be highly optimized for its specific task, leading to better performance and modularity.

We will demonstrate this by building a system with three distinct specialists:
1.  **Generalist Agent:** Handles casual conversation and simple questions.
2.  **Research Agent:** Equipped with a search tool to answer questions about recent events or complex topics.
3.  **Coding Agent:** A specialist focused on generating Python code snippets.

The Meta-Controller will be the "brain" of the operation, examining each user query and deciding which agent is best suited to respond. This creates a flexible and easily extensible system where new capabilities can be added simply by creating a new specialist agent and teaching the controller about it.

### Definition
A **Meta-Controller** (or Router) is a supervisory agent in a multi-agent system that is responsible for analyzing incoming tasks and dispatching them to the appropriate specialized sub-agent or workflow. It acts as an intelligent routing layer, deciding which tool or expert is best suited for the job at hand.

### High-level Workflow

1.  **Receive Input:** The system receives a user request.
2.  **Meta-Controller Analysis:** The Meta-Controller agent examines the request's intent, complexity, and content.
3.  **Dispatch to Specialist:** Based on its analysis, the Meta-Controller selects the best specialist agent (e.g., 'Researcher', 'Coder', 'Chatbot') from a predefined pool.
4.  **Execute Task:** The chosen specialist agent executes the task and generates a result.
5.  **Return Result:** The result from the specialist is returned to the user. In more complex workflows, control might return to the Meta-Controller for further steps or monitoring.

### When to Use / Applications
*   **Multi-Service AI Platforms:** A single entry point for a platform that offers diverse services like document analysis, data visualization, and creative writing.
*   **Adaptive Personal Assistants:** An assistant that can switch between different modes or tools, such as managing your calendar, searching the web, or controlling smart home devices.
*   **Enterprise Workflows:** Routing customer support tickets to the right department (technical, billing, sales) based on the ticket's content.

### Strengths & Weaknesses
*   **Strengths:**
    *   **Flexibility & Modularity:** Extremely easy to add new capabilities by simply adding a new specialist agent and updating the controller's routing logic.
    *   **Performance:** Allows for highly optimized, expert agents instead of one jack-of-all-trades model that might be mediocre at everything.
*   **Weaknesses:**
    *   **Controller as a Single Point of Failure:** The quality of the entire system hinges on the controller's ability to route tasks correctly. A poor routing decision leads to a suboptimal or incorrect outcome.
    *   **Potential for Increased Latency:** The extra step of routing can add a small amount of latency compared to a direct call to a single agent.

## Phase 0: Foundation & Setup

We'll install libraries and set up our environment. We'll need `langchain-tavily` for our Research Agent's search tool.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

In [2]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_tavily import TavilySearch
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Meta-Controller (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## Phase 1: Building the Specialist Agents

First, we'll create our team of expert agents. Each will be a simple chain with a specific persona and, in the case of the Researcher, a tool. We will wrap them in a node function for use in our LangGraph.

In [3]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
search_tool = TavilySearch(max_results=3)

# Define the state for the overall graph
class MetaAgentState(TypedDict):
    user_request: str
    next_agent_to_call: Optional[str]
    generation: str

# A helper factory function to create specialist agent nodes
def create_specialist_node(persona: str, tools: list = None):
    """Factory to create a specialist agent node."""
    system_prompt = f"You are a specialist agent with the following persona: {persona}. Respond directly and concisely to the user's request based on your role."
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{user_request}")
    ])
    
    if tools:
        chain = prompt | llm.bind_tools(tools)
    else:
        chain = prompt | llm
        
    def specialist_node(state: MetaAgentState) -> Dict[str, Any]:
        result = chain.invoke({"user_request": state['user_request']})
        return {"generation": result.content}
    
    return specialist_node

# 1. Generalist Agent Node
generalist_node = create_specialist_node(
    "You are a friendly and helpful generalist AI assistant. You handle casual conversation and simple questions."
)

# 2. Research Agent Node
research_agent_node = create_specialist_node(
    "You are an expert researcher. You must use your search tool to find information to answer the user's question.",
    tools=[search_tool]
)

# 3. Coding Agent Node
coding_agent_node = create_specialist_node(
    "You are an expert Python programmer. Your task is to write clean, efficient Python code based on the user's request. Provide only the code, wrapped in markdown code blocks, with minimal explanation."
)

print("Specialist agents defined successfully.")

Specialist agents defined successfully.


## Phase 2: Building the Meta-Controller

This is the brain of our system. The Meta-Controller is an LLM-powered node whose only job is to decide which specialist to route the request to. The quality of its prompt is critical for the system's performance.

In [4]:
# Pydantic model for the controller's routing decision
class ControllerDecision(BaseModel):
    next_agent: str = Field(description="The name of the specialist agent to call next. Must be one of ['Generalist', 'Researcher', 'Coder'].")
    reasoning: str = Field(description="A brief reason for choosing the next agent.")

def meta_controller_node(state: MetaAgentState) -> Dict[str, Any]:
    """The central controller that decides which specialist to call."""
    console.print("--- 🧠 Meta-Controller Analyzing Request ---")
    
    # Define the specialists and their descriptions for the controller
    specialists = {
        "Generalist": "Handles casual conversation, greetings, and simple questions.",
        "Researcher": "Answers questions about recent events, complex topics, or anything requiring up-to-date information from the web.",
        "Coder": "Writes Python code based on a user's specification."
    }
    
    specialist_descriptions = "\n".join([f"- {name}: {desc}" for name, desc in specialists.items()])
    
    prompt = ChatPromptTemplate.from_template(
        f"""You are the meta-controller for a multi-agent AI system. Your job is to analyze the user's request and route it to the most appropriate specialist agent.

Here are the available specialists:
{specialist_descriptions}

Analyze the following user request and choose the best specialist to handle it. Provide your decision in the required format.

User Request: "{{user_request}}""""
    )
    
    controller_llm = llm.with_structured_output(ControllerDecision)
    chain = prompt | controller_llm
    
    decision = chain.invoke({"user_request": state['user_request']})
    console.print(f"[yellow]Routing decision:[/yellow] Send to [bold]{decision.next_agent}[/bold]. [italic]Reason: {decision.reasoning}[/italic]")
    
    return {"next_agent_to_call": decision.next_agent}

print("Meta-Controller node defined successfully.")

Meta-Controller node defined successfully.


## Phase 3: Assembling and Running the Graph

Now we'll use LangGraph to wire everything together. The graph will start with the Meta-Controller, and then, based on its decision, a conditional edge will route the state to the correct specialist node. After the specialist runs, the graph will end.

In [5]:
# Build the graph
workflow = StateGraph(MetaAgentState)

# Add nodes for the controller and each specialist
workflow.add_node("meta_controller", meta_controller_node)
workflow.add_node("Generalist", generalist_node)
workflow.add_node("Researcher", research_agent_node)
workflow.add_node("Coder", coding_agent_node)

# Set the entry point
workflow.set_entry_point("meta_controller")

# Define the conditional routing logic
def route_to_specialist(state: MetaAgentState) -> str:
    """Reads the controller's decision and returns the name of the node to route to."""
    return state["next_agent_to_call"]

workflow.add_conditional_edges(
    "meta_controller",
    route_to_specialist,
    {
        "Generalist": "Generalist",
        "Researcher": "Researcher",
        "Coder": "Coder"
    }
)

# After any specialist runs, the process ends
workflow.add_edge("Generalist", END)
workflow.add_edge("Researcher", END)
workflow.add_edge("Coder", END)

meta_agent = workflow.compile()
print("Meta-Controller agent graph compiled successfully.")

Meta-Controller agent graph compiled successfully.


## Phase 4: Demonstration

Let's test our Meta-Controller with a variety of prompts to see if it correctly dispatches them to the right specialist.

In [6]:
def run_agent(query: str):
    result = meta_agent.invoke({"user_request": query})
    console.print("\n[bold]Final Response:[/bold]")
    console.print(Markdown(result['generation']))

# Test 1: Should be routed to the Generalist
console.print("--- 💬 Test 1: General Conversation ---")
run_agent("Hello, how are you today?")

# Test 2: Should be routed to the Researcher
console.print("\n--- 🔬 Test 2: Research Question ---")
run_agent("What were NVIDIA's latest financial results?")

# Test 3: Should be routed to the Coder
console.print("\n--- 💻 Test 3: Coding Request ---")
run_agent("Can you write me a python function to calculate the nth fibonacci number?")

--- 💬 Test 1: General Conversation ---


--- 🧠 Meta-Controller Analyzing Request ---
Routing decision: Send to Generalist. Reason: The user's request is a simple greeting, which falls under the category of casual conversation handled by the Generalist agent.



Final Response:


Hello there! How can I help you today?


--- 🔬 Test 2: Research Question ---


...

NVIDIA's latest financial results, for the quarter ending in April 2024, were exceptionally strong. They reported revenue of $26.04 billion, a significant increase year-over-year, driven largely by their Data Center revenue which hit a record $22.6 billion. Their GAAP earnings per diluted share were $5.98.


--- 💻 Test 3: Coding Request ---


--- 🧠 Meta-Controller Analyzing Request ---
Routing decision: Send to Coder. Reason: The user is explicitly asking for a Python function, which is a coding task. The Coder agent is the specialist for this.



Final Response:


```python
def fibonacci(n):
    """Calculates the nth Fibonacci number."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b
```

## Conclusion

In this notebook, we have successfully implemented a **Meta-Controller** architecture. Our tests clearly demonstrate its primary function: acting as an intelligent and dynamic router.

1.  The simple greeting was correctly identified and sent to the **Generalist**.
2.  The query about recent financial news was dispatched to the **Researcher**, which used its search tool to fetch up-to-date information.
3.  The request for a code snippet was routed to the **Coder**, which provided a well-formatted and correct function.

This pattern is exceptionally powerful for building scalable and maintainable AI systems. By separating concerns, each specialist can be improved independently without affecting the others. The system's overall intelligence can be enhanced simply by adding new, more capable specialists and making the Meta-Controller aware of them. While the controller itself represents a potential bottleneck, its role as a flexible orchestrator is a cornerstone of advanced agentic design.